# **응급상황 자동 인식 및 응급실 연계 서비스**
# **단계1 : 응급상황 음성 인식 및 요약**

## **0.미션**

단계 1에서는, 응급상황의 음성을 인식해서 텍스트로 변환하고, 변환된 텍스트를 다시 요약 및 핵심키워드 도출 작업을 수행합니다.  
이를 위해 사전학습된 모델을 API로 연결하여 활용합니다.

### (1) 미션1
* 음성인식 : STT(Speech-to-Text)
    * 사용 모델 : OpenAI의 **Whisper-1**
    * 제공받은 음성 파일과 새로 제작하는 5건 이상의 음성파일을 텍스트로 변환하고, 변환작업이 잘 되는지 확인해 봅시다.

### (2) 미션2
* 텍스트 요약 및 핵심 키워드 도출
    * 사용 모델 : OpenAI의 **GPT-3.5-turbo**
    * 내용 요약과 주요 키워드를 도출하도록
    프롬프트 입력과 출력을 구성하고 테스트 해 봅시다.

* [추가]응급실 현황 다운로드(이 데이터는 단계3에서 필요합니다.)



## **1.환경설정**

### (1) 경로 설정

구글 드라이브 연결

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(project6_2)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/project6_2/'

### (2) 라이브러리

#### 1) 필요한 라이브러리 설치

* requirements.txt 파일의 [경로 복사]를 한 후,
* 아래 경로에 붙여 넣기

In [ ]:
!pip install -r path = /content/drive/MyDrive/project6_2/requirements.txt -q

ERROR: Invalid requirement: '=': Expected package name at the start of dependency specifier
    =
    ^
Hint: = is not a valid operator. Did you mean == ?


#### 2) 라이브러리 로딩

In [ ]:
#필요한 라이브러리 설치 및 불러오기
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import matplotlib.pyplot as plt
import openai
from openai import OpenAI
import json

# 더 필요한 라이브러리 추가 -------------

### (3) OpenAI API Key 환경 변수 설정

* 제공받은 open ai api key를 **api_key.txt** 파일에 저장합니다.
    * (제공받은 api_key.txt 파일은 비어 있습니다.)

* 다음 코드를 통해 환경변수로 등록 합니다.

In [ ]:
def load_file(filepath):
    with open(filepath, 'r') as file:
        return file.readline().strip()

# API 키 로드 및 환경변수 설정
openai.api_key = load_file(path + 'api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key

* ⚠️ 아래 코드셀은, 실행해서 key가 제대로 보이는지 확인하고 결과는 삭제하세요.

In [ ]:
print(os.environ['OPENAI_API_KEY'])

## **2. 미션1 : STT**

### (1) 제공된 데이터 변환
* 세부사항
    * 사용 모델 : whisper-1
    * 제공 받은 오디오 파일을 읽어서 텍스트로 변환시켜 봅시다.
        * 반복문을 통해 파일 하나씩 읽어서 텍스트 변환
        * 변환된 텍스트를 데이터 프레임에 추가

|filename|text|
|----|----|
|audio3.mp3|어쩌구 저쩌구...급해요.|

* 음성파일 변환

In [ ]:
# 음성파일 경로 지정
audio_path = path + 'audio/temp/'

In [ ]:
# OpenAI 클라이언트 생성
client = OpenAI()

In [ ]:
# 위스퍼 모델 사용 : 제공된 음성파일 중 1개를 텍스트로 변환해보기
filename = 'audio2.mp3'
audio_file = open(audio_path + filename, "rb")
transcript = client.audio.transcriptions.create(
    file=audio_file,
    model="whisper-1",
    language="ko",
    response_format="text",
)

print(transcript, type(transcript))

119죠. 제가 지금 열이 열이 올랐어요. 몇 도냐면은 38도 정도 돼요. 머리가 아프고 좀 띵한 것 같아요. 우한이 좀 들어요. 어떻게 해야 할까요?
 <class 'str'>


* 음성파일 변환 함수 생성

In [ ]:
def audio_to_text(audio_path, filename):
    # OpenAI 클라이언트 생성
    client = OpenAI()

    # 오디오 파일을 읽어서, 위스퍼를 사용한 변환
    audio_file = open(audio_path + filename, "rb")
    transcript = client.audio.transcriptions.create(
        file=audio_file,
        model="whisper-1",
        language="ko",
        response_format="text",
    )

    # 변환된 텍스트를 반환
    print(transcript, type(transcript))

    # 결과 반환
    return transcript

In [ ]:
# 음성파일 이름을 리스트에 담기
file_names = [f for f in os.listdir(audio_path) if os.path.isfile(os.path.join(audio_path, f))]
print(file_names)

['audio2.mp3', 'audio5.mp3', 'audio3.mp3', 'audio1.mp3', 'audio4.mp3']


In [ ]:
# 반복문을 통해, 파일 하나씩 읽어서 텍스트 변환, 변환된 텍스트를 데이터 프레임에 추가

# 빈 데이터프레임 선언
df = pd.DataFrame(columns=['filename', 'text'])

# 반복문 수행하면서 오디오 변환
for filename in file_names:
    transcript = audio_to_text(audio_path, filename)

    # 변환된 텍스트를 데이터프레임에 추가
    row = pd.DataFrame({'filename': [filename], 'text': [transcript]})
    df = pd.concat([df, row], ignore_index=True)

# 데이터프레임 결과 조회
df.head()

119죠. 제가 지금 열이 열이 올랐어요. 몇 도냐면은 38도 정도 돼요. 머리가 아프고 좀 띵한 것 같아요. 우한이 좀 들어요. 어떻게 해야 할까요?
 <class 'str'>
화장실에서 미끄러워서 엉덩방아를 찍었어요. 그러고 꼬리뼈가 계속 아파요. 점점 아픈 것 같은데 응급실을 가야 할까요?
 <class 'str'>
동생이 콩 가지고 놀다가 코에 들어가서 한쪽 코가 막혔어요. 아무리 빼보려 해도 안 빠져요. 어떻게 해야 할까요? 동생이 너무 힘들어 하네요.
 <class 'str'>
지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어요. 근데 조금 어지럽다고 하네요. 네네 계단에서 굴렀어요. 지금은 물 마시고 있는데 이거 응급실로 가봐야 할까요? 피도 지금 머졌어요. 네네 나이는 마흔아홉 살 이세요. 어떻게 해야 할지 모르겠어요.
 <class 'str'>
아까 가다가 머리를 박았는데, 처음에는 괜찮다가, 지금 3시간 정도 지났는데, 머리가 어지럽고 속이 매스꺼워요. 어떻게 해야 할까요?
 <class 'str'>


,filename,text
0,audio2.mp3,119죠. 제가 지금 열이 열이 올랐어요. 몇 도냐면은 38도 정도 돼요. 머리가 ...
1,audio5.mp3,화장실에서 미끄러워서 엉덩방아를 찍었어요. 그러고 꼬리뼈가 계속 아파요. 점점 아픈...
2,audio3.mp3,동생이 콩 가지고 놀다가 코에 들어가서 한쪽 코가 막혔어요. 아무리 빼보려 해도 안...
3,audio1.mp3,지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어...
4,audio4.mp3,"아까 가다가 머리를 박았는데, 처음에는 괜찮다가, 지금 3시간 정도 지났는데, 머리..."


### (2) 오디오 데이터 추가 수집(제작) 및 변환

* 세부사항
    * 응급 상황에 맞는 음성 녹음하기
        * 응급 등급별 2개 이상씩(총 10개 이상)
    * 반복문을 통해 모든 음성 파일 데이터 변환 : STT
        * 변환 내용은 위에서 저장한 데이터프레임에 추가
    * 변환 후 음성 내용과 변환 결과를 비교


In [ ]:
# 음성파일 경로 지정
audio_path = path + 'audio/'

In [ ]:
# 음성파일 이름을 리스트에 담기
file_names = [f for f in os.listdir(audio_path) if os.path.isfile(os.path.join(audio_path, f))]
print(file_names)

['level_2_1.m4a', 'level_1_1.m4a', 'level_4_1.m4a', 'level_5_2.m4a', 'level_3_2.m4a', 'level_3_1.m4a', 'level_1_2.m4a', 'level_5_1.m4a', 'level_4_2.m4a', 'level_2_2.m4a']


In [ ]:
# 반복문 수행하면서 오디오 변환
for filename in file_names:
    transcript = audio_to_text(audio_path, filename)

    # 변환된 텍스트를 데이터프레임에 추가
    row = pd.DataFrame({'filename': [filename], 'text': [transcript]})
    df = pd.concat([df, row], ignore_index=True)

# 데이터프레임 결과 조회
df

아들이 피를 토하고 있어요. 출혈이 너무 심해서 지금 바로 병원에 가야 할 것 같아요.
 <class 'str'>
저희 아버지가 지금 음식을 삼키지도 못하고 숨쉬기도 힘들어 하는데 의식이 없어요. 빨리 병원으로 가야 할 것 같습니다.
 <class 'str'>
저희 아버지가 저체온증에 의심돼요. 동상은 없어요. 체온은 35도 정도 되는 것 같고 숨은 잘 쉬세요. 그래도 확인을 받고 싶어요.
 <class 'str'>
저희 남편이 화상을 입었는데 통증이 가벼워서 크게 걱정되지는 않아요. 병원에 가는 게 좋을까요?
 <class 'str'>
아이가 락스를 조금 먹은 것 같아요. 호흡곤란은 없어요. 그래도 병원에 가봐야 할까요?
 <class 'str'>
저희 와이프가 갑각류 알레르기 반응으로 숨을 조금 가빠해요. 응급처치가 필요할 것 같은데 괜찮을까요?
 <class 'str'>
어머니가 목이 너무 아프시고 숨을 제대로 쉬지 못하세요. 지금 상태가 너무 안 좋으셔서 빨리 도와주세요.
 <class 'str'>
아이가 강아지한테 물렸는데 통증이 그렇게 심해 보이진 않아요. 그런데 더 악화되진 않을지 걱정돼요.
 <class 'str'>
어머니 피부에 발진이 생겼는데 별로 아프시지는 않으시고 상태가 그렇게 심각하진 않아요. 그래도 병원에 가야 할까요?
 <class 'str'>
친구가 사탕이 목에 걸린 것 같은데 숨을 제대로 쉬지 못하고 있어요
 <class 'str'>


,filename,text
0,audio2.mp3,119죠. 제가 지금 열이 열이 올랐어요. 몇 도냐면은 38도 정도 돼요. 머리가 ...
1,audio5.mp3,화장실에서 미끄러워서 엉덩방아를 찍었어요. 그러고 꼬리뼈가 계속 아파요. 점점 아픈...
2,audio3.mp3,동생이 콩 가지고 놀다가 코에 들어가서 한쪽 코가 막혔어요. 아무리 빼보려 해도 안...
3,audio1.mp3,지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어...
4,audio4.mp3,"아까 가다가 머리를 박았는데, 처음에는 괜찮다가, 지금 3시간 정도 지났는데, 머리..."
5,level_2_1.m4a,아들이 피를 토하고 있어요. 출혈이 너무 심해서 지금 바로 병원에 가야 할 것 같아...
6,level_1_1.m4a,저희 아버지가 지금 음식을 삼키지도 못하고 숨쉬기도 힘들어 하는데 의식이 없어요. ...
7,level_4_1.m4a,저희 아버지가 저체온증에 의심돼요. 동상은 없어요. 체온은 35도 정도 되는 것 같...
8,level_5_2.m4a,저희 남편이 화상을 입었는데 통증이 가벼워서 크게 걱정되지는 않아요. 병원에 가는 ...
9,level_3_2.m4a,아이가 락스를 조금 먹은 것 같아요. 호흡곤란은 없어요. 그래도 병원에 가봐야 할까...


## **3. 미션2 : Summary**

* 세부사항
    * 문서요약 예제 파일을 참조하여 테스트 해 봅니다.
    * 코드를 참조하여, 원하는 형식에 맞게 요약이 되도록 프롬프트를 구성합니다.
        * 요약 시 중요 키워드들이 함께 도출되도록 합니다.
        * 가능하다면, 요약 문장 길이에 제한을 둡시다.
    * 반복문을 통해 요약하고, 결과를 데이터프레임에 추가합니다.
        * summary 열을 추가하고, 요약 결과를 입력
            * 요약결과와 키워드는 하나의 문자열로 붙여서 summary열에 추가

### (1) 문서 요약

* 문서 요약 함수로 생성

In [ ]:
def text_summary(input_text):
    # OpenAI 클라이언트 생성
    client = OpenAI()

    # 시스템 역할과 응답 형식 지정
    system_role = '''당신은 전화 내용을 듣고 환자의 상태를 요약해 전달해주는 어시스턴트입니다.
    환자의 중증도를 잘 분류할 수 있도록 단서를 알려주세요.
    환자의 상태를 제외한 주관적인 판단 내용은 절대 말하지 않습니다.
    "환자가 ~한 상태"로 응답을 마쳐주세요.
    응답은 다음의 형식을 지켜주세요.
    {"summary": \"텍스트 요약\"}
    '''

    # 입력데이터를 GPT-3.5-turbo에 전달하고 답변 받아오기
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": system_role
            },
            {
                "role": "user",
                "content": input_text
            }
        ]
    )

    # 응답 받기
    answer = response.choices[0].message.content

    # 응답형식을 정리하고 return
    answer = json.loads(answer)
    print(answer['summary'])
    return answer['summary']

* 저장된 text를 하나씩 불러와서 요약하고 다시 저장하기

In [ ]:
df['summary'] = df['text'].apply(text_summary)

환자가 열이 38도로 올라가며 머리가 아프고 띵한 상태이고, 목에 통증을 느끼고 있습니다.
환자가 화장실에서 미끄러져 엉덩방아를 찍었고, 꼬리뼈가 계속 아프다고 합니다. 점점 아픔이 느껴진다고 합니다.
환자가 콩으로 인해 한쪽 코가 막혀 있고, 콩을 빼려 해도 제거되지 않아 호흡에 불편함을 느낍니다.
환자가 계단에서 넘어져 머리에서 피가 나며 숨을 쉬고 있습니다. 일어나자마자 어지러움을 느낀 상태이고 지금은 물을 마시고 있습니다. 응급실 방문이 필요한 상황입니다.
환자가 머리를 박고 3시간이 지났는데, 머리가 어지럽고 속이 메스꺼운 상태입니다. 의료진 상담이 필요합니다.
환자가 심한 출혈으로 피를 토하고 있는 상태입니다. 긴급하게 병원에 가야할 것으로 보입니다.
환자가 음식을 삼키지 못하고 숨쉬기도 힘들어하는 상태이며 의식이 없어서 긴급한 의료 도움이 필요한 상태입니다.
환자가 저체온증 가능성이 있으며, 체온이 약 35도이고 숨을 잘 쉬고 있는 상태입니다. 확인을 받기 위해 의료진을 방문하고 싶어합니다.
환자가 화상을 입어 통증이 가벼운 상태입니다. 병원 방문은 필요하지 않을 수 있습니다.
아이가 락스를 섭취한 상황이며 호흡곤란은 없는 상태이나 병원 방문이 필요합니다.
환자가 갑각류 알레르기 반응으로 숨을 조금 가빠한 상태입니다. 응급처치가 필요할 수 있습니다.
환자가 목이 매우 아프고 숨을 제대로 쉬지 못하는 상태이며, 상황이 심각하여 긴급한 도움이 필요합니다.
아이가 강아지에게 물린 상처가 있지만 통증이 심하게 보이지는 않음. 악화될 가능성에 대해 걱정 중.
환자가 발진이 생겼지만 아프지 않고 중증도가 높지 않다고 합니다. 그러나 병원 방문을 권유하는 것이 좋을 것으로 보입니다.
환자가 사탕이 목에 걸린 상태로 숨을 제대로 쉬지 못하고 있습니다.


In [ ]:
df

,filename,text,summary
0,audio2.mp3,119죠. 제가 지금 열이 열이 올랐어요. 몇 도냐면은 38도 정도 돼요. 머리가 ...,"환자가 열이 38도로 올라가며 머리가 아프고 띵한 상태이고, 목에 통증을 느끼고 있..."
1,audio5.mp3,화장실에서 미끄러워서 엉덩방아를 찍었어요. 그러고 꼬리뼈가 계속 아파요. 점점 아픈...,"환자가 화장실에서 미끄러져 엉덩방아를 찍었고, 꼬리뼈가 계속 아프다고 합니다. 점점..."
2,audio3.mp3,동생이 콩 가지고 놀다가 코에 들어가서 한쪽 코가 막혔어요. 아무리 빼보려 해도 안...,"환자가 콩으로 인해 한쪽 코가 막혀 있고, 콩을 빼려 해도 제거되지 않아 호흡에 불..."
3,audio1.mp3,지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어...,환자가 계단에서 넘어져 머리에서 피가 나며 숨을 쉬고 있습니다. 일어나자마자 어지러...
4,audio4.mp3,"아까 가다가 머리를 박았는데, 처음에는 괜찮다가, 지금 3시간 정도 지났는데, 머리...","환자가 머리를 박고 3시간이 지났는데, 머리가 어지럽고 속이 메스꺼운 상태입니다. ..."
5,level_2_1.m4a,아들이 피를 토하고 있어요. 출혈이 너무 심해서 지금 바로 병원에 가야 할 것 같아...,환자가 심한 출혈으로 피를 토하고 있는 상태입니다. 긴급하게 병원에 가야할 것으로 ...
6,level_1_1.m4a,저희 아버지가 지금 음식을 삼키지도 못하고 숨쉬기도 힘들어 하는데 의식이 없어요. ...,환자가 음식을 삼키지 못하고 숨쉬기도 힘들어하는 상태이며 의식이 없어서 긴급한 의료...
7,level_4_1.m4a,저희 아버지가 저체온증에 의심돼요. 동상은 없어요. 체온은 35도 정도 되는 것 같...,"환자가 저체온증 가능성이 있으며, 체온이 약 35도이고 숨을 잘 쉬고 있는 상태입니..."
8,level_5_2.m4a,저희 남편이 화상을 입었는데 통증이 가벼워서 크게 걱정되지는 않아요. 병원에 가는 ...,환자가 화상을 입어 통증이 가벼운 상태입니다. 병원 방문은 필요하지 않을 수 있습니다.
9,level_3_2.m4a,아이가 락스를 조금 먹은 것 같아요. 호흡곤란은 없어요. 그래도 병원에 가봐야 할까...,아이가 락스를 섭취한 상황이며 호흡곤란은 없는 상태이나 병원 방문이 필요합니다.


### (2) 전국 병원 응급실 정보 수집



#### 1) 인증키 발급

* 인증키 발급 절차
    * 1) data.go.kr 회원가입
    * 2) 국립중앙의료원_전국 응급의료기관 정보 조회 서비스
https://www.data.go.kr/data/15000563/openapi.do 로 이동
    * 3) 활용신청
        * 활용목적 : 기타(개인 학습 용도)
        * 상세 기능선택
            * 응급의료기관 목록정보 조회
            * 응급의료기관 위치정보 조회
            * 응급의료기관 기본정보 조회
    * 4) 인증키 확인
        * 마이페이지 > Open API > 활용신청현황
        * [승인] 국립중앙의료원_전국 응급의료기관 정보 조회 서비스
        * 일반 인증키(Decoding) 이용

#### 2) 데이터 수집

In [ ]:
# path 확인
path

'/content/drive/MyDrive/AIVLE/프로젝트/6차/6-2/'

In [ ]:
# 응급실 데이터 수집하기

url = 	'http://apis.data.go.kr/B552657/ErmctInfoInqireService/getEgytListInfoInqire'
serviceKey = '9v5SY9vv2bgq8iGVPmrjPWocUvAlA9OrjK73ix6cPAFEgyqoaNrSUrBMs+bYWIpjG7k81WWhfWYe/KJHvT0iXQ==' # 여러분의 일반 인증키(Decoding)

params = {
    'serviceKey': serviceKey,
    'pageNo': '1', 'numOfRows': '1000',  # 전체 응급실 수가 500여개 됨. 1000개면 충분
    'format': 'xml'
}

response = requests.get(url, params = params)

# 정상 수행 되었다면 200
print(response)

<Response [200]>


In [ ]:
# response xml에서 주요 정보 찾기
root = ET.fromstring(response.text)

data = []

for item in root. findall('.//item'):
        dutyName = item.findtext('dutyName') # 병원 이름
        dutyAddr = item.findtext('dutyAddr') # 병원 주소
        dutyTel1 = item.findtext('dutyTel1') # 병원 전화번호
        dutyTel3 = item.findtext('dutyTel3') # 응급실 전화번호
        dutyEmclsName = item.findtext('dutyEmclsName') # 응급의료기관 종류
        wgs84Lon = item.findtext('wgs84Lon') # 병원 경도
        wgs84Lat = item.findtext('wgs84Lat') # 병원 위도

        # 빈 리스트 data에 딕시너리 형태({'칼럼이름':값, ...})로 저장(추가)
        data.append({
            'Hospital': dutyName,
            'Address': dutyAddr,
            'Telephone': dutyTel3 if dutyTel3 else dutyTel1, # 응급실 전화번호가 없으면 병원 전화번호를 사용
            "Medical_Type": dutyEmclsName,
            'Longitude': wgs84Lon,
            'Latitude': wgs84Lat
        })

# 데이터프레임으로 변환
df_hospital = pd.DataFrame(data)

csv 파일에서 추출

In [ ]:
# csv 파일로 저장(인덱스 제외)
df_hospital.to_csv(path + 'df_hospital.csv', index=False)

## **Mission Complete!**

수고 많았습니다!